In [1]:
import matplotlib.pylab as plt
from sklearn.metrics import classification_report

from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, GlobalMaxPooling2D
from keras.optimizers import Adam

import tensorflow as tf
from keras import backend as K
from keras import initializers

import numpy as np
import random

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from utils import *
from data import *

/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
random.seed(42)
np.random.seed(42)
tf.set_random_seed(42)
my_init = initializers.glorot_uniform(seed=42)

In [4]:
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [5]:
points = load_keypoints_txt_from_folders("./facs_me_db/Landmarks/")
images = load_photos_from_folders("./facs_me_db/cohn-kanade-images/")
labels = load_label_txt_from_folders("./facs_me_db/Emotion/")
labels2 = load_label_txt_from_folders("./facs_me_db/FACS_labels/FACS/")

['S005', '001', '00000001.png'] Loaded
['S005', '001', '00000002.png'] Loaded
['S005', '001', '00000003.png'] Loaded
['S005', '001', '00000004.png'] Loaded
['S005', '001', '00000005.png'] Loaded
['S005', '001', '00000006.png'] Loaded
['S005', '001', '00000007.png'] Loaded
['S005', '001', '00000008.png'] Loaded
['S005', '001', '00000009.png'] Loaded
['S005', '001', '00000010.png'] Loaded
['S005', '001', '00000011.png'] Loaded
['S010', '001', '00000001.png'] Loaded
['S010', '001', '00000002.png'] Loaded
['S010', '001', '00000003.png'] Loaded
['S010', '001', '00000004.png'] Loaded
['S010', '001', '00000005.png'] Loaded
['S010', '001', '00000006.png'] Loaded
['S010', '001', '00000007.png'] Loaded
['S010', '001', '00000008.png'] Loaded
['S010', '001', '00000009.png'] Loaded
['S010', '001', '00000010.png'] Loaded
['S010', '001', '00000011.png'] Loaded
['S010', '001', '00000012.png'] Loaded
['S010', '001', '00000013.png'] Loaded
['S010', '001', '00000014.png'] Loaded
['S010', '002', '00000001

In [6]:
X, Y1, Y2, Y3 = make_x_y(points, images, labels, labels2)

In [7]:
X_train, X_test, Y_train, Y_test, Y_train2, Y_test2, Y_train3, Y_test3 = prepare_train_test(X, Y1, Y2, Y3, W)
# 1: emotions
# 2: FACS
# 3: facial keypoints

In [8]:
input_shape = (W, W, 1)

# Plain CNN with a single output

In [9]:
visible = Input(shape=input_shape)
x = Conv2D(10, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(visible)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(20, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(30, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(x)
x = GlobalMaxPooling2D()(x)
output = Dense(Y_train.shape[1], activation='softmax', kernel_initializer=my_init)(x)
model = Model(inputs=visible, outputs=output)

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50, 50, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 50, 10)        100       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 10)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 20)        1820      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 20)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 30)        5430      
_________________________________________________________________
global_max_pooling2d_1 (Glob (None, 30)                0         
__________

In [11]:
model.compile(loss=['categorical_crossentropy'],
              optimizer=Adam(clipnorm = 1.), metrics = ['accuracy'])

In [12]:
model.fit(X_train, Y_train,
          validation_data = (X_test, Y_test),
          batch_size = 64,
          epochs = 10,
          verbose = True,
          shuffle = False)

Train on 1982 samples, validate on 661 samples
Epoch 1/10
1982/1982 [==============================] - 9s 5ms/step - loss: 10.1771 - acc: 0.2013 - val_loss: 9.7857 - val_acc: 0.2617
Epoch 2/10
1982/1982 [==============================] - 8s 4ms/step - loss: 9.8479 - acc: 0.2225 - val_loss: 9.7958 - val_acc: 0.2073
Epoch 3/10
1982/1982 [==============================] - 6s 3ms/step - loss: 9.6836 - acc: 0.2412 - val_loss: 9.5888 - val_acc: 0.2511
Epoch 4/10
1982/1982 [==============================] - 6s 3ms/step - loss: 9.5785 - acc: 0.2593 - val_loss: 9.4756 - val_acc: 0.2738
Epoch 5/10
1982/1982 [==============================] - 6s 3ms/step - loss: 8.0342 - acc: 0.2810 - val_loss: 5.3949 - val_acc: 0.3268
Epoch 6/10
1982/1982 [==============================] - 6s 3ms/step - loss: 5.3215 - acc: 0.3431 - val_loss: 5.0127 - val_acc: 0.3570
Epoch 7/10
1982/1982 [==============================] - 6s 3ms/step - loss: 4.2963 - acc: 0.3507 - val_loss: 3.6033 - val_acc: 0.4009
Epoch 8/10
198

In [13]:
pred = model.predict(X_test)
y_pred = [np.argmax(p) for p in pred]
y_true = [np.argmax(p) for p in Y_test]

print classification_report(y_true, y_pred)

             precision    recall  f1-score   support

          0       0.39      0.56      0.46       149
          1       0.00      0.00      0.00        10
          2       0.00      0.00      0.00        71
          3       0.32      0.11      0.16        74
          4       0.47      0.64      0.54       163
          5       0.44      0.11      0.17        64
          6       0.38      0.54      0.44       130

avg / total       0.36      0.41      0.36       661



/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# CNN with auxiliary FACS output

In [14]:
visible = Input(shape=input_shape)
x = Conv2D(10, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(visible)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(20, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(30, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(x)
x = GlobalMaxPooling2D()(x)
output = Dense(Y_train.shape[1], activation='softmax', name = 'emotion',
               kernel_initializer=my_init)(x)
output2 = Dense(Y_train2.shape[1], activation='sigmoid', name = 'facs',
                kernel_initializer=my_init)(x)
model = Model(inputs=visible, outputs=[output, output2])

In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 50, 50, 1)    0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 50, 50, 10)   100         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 25, 25, 10)   0           conv2d_4[0][0]                   
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 25, 25, 20)   1820        max_pooling2d_3[0][0]            
__________________________________________________________________________________________________
max_poolin

In [16]:
model.compile(loss=['categorical_crossentropy', 'binary_crossentropy'],
              optimizer=Adam(clipnorm = 1.), metrics = {'emotion': 'accuracy'}, loss_weights = [1, 1])

In [17]:
model.fit(X_train, [Y_train, Y_train2],
          validation_data = (X_test, [Y_test, Y_test2]),
          batch_size = 64,
          epochs = 10,
          verbose = True,
          shuffle = False)

Train on 1982 samples, validate on 661 samples
Epoch 1/10
1982/1982 [==============================] - 7s 3ms/step - loss: 12.8519 - emotion_loss: 10.3772 - facs_loss: 2.4747 - emotion_acc: 0.2175 - val_loss: 12.0842 - val_emotion_loss: 10.2146 - val_facs_loss: 1.8696 - val_emotion_acc: 0.1422
Epoch 2/10
1982/1982 [==============================] - 7s 4ms/step - loss: 9.0798 - emotion_loss: 7.9094 - facs_loss: 1.1704 - emotion_acc: 0.2503 - val_loss: 6.0267 - val_emotion_loss: 5.2797 - val_facs_loss: 0.7471 - val_emotion_acc: 0.3298
Epoch 3/10
1982/1982 [==============================] - 6s 3ms/step - loss: 3.7467 - emotion_loss: 3.1165 - facs_loss: 0.6302 - emotion_acc: 0.2977 - val_loss: 2.1931 - val_emotion_loss: 1.6727 - val_facs_loss: 0.5205 - val_emotion_acc: 0.3737
Epoch 4/10
1982/1982 [==============================] - 6s 3ms/step - loss: 2.0145 - emotion_loss: 1.5851 - facs_loss: 0.4294 - emotion_acc: 0.3794 - val_loss: 1.8435 - val_emotion_loss: 1.4654 - val_facs_loss: 0.3781

In [18]:
pred = model.predict(X_test)[0]
y_pred = [np.argmax(p) for p in pred]
y_true = [np.argmax(p) for p in Y_test]

print classification_report(y_true, y_pred)

             precision    recall  f1-score   support

          0       0.58      0.36      0.44       149
          1       0.00      0.00      0.00        10
          2       0.47      0.27      0.34        71
          3       0.36      0.65      0.46        74
          4       0.56      0.83      0.67       163
          5       0.56      0.08      0.14        64
          6       0.62      0.68      0.65       130

avg / total       0.53      0.53      0.49       661



# CNN with auxiliary KEYPOINT output

In [19]:
visible = Input(shape=input_shape)
x = Conv2D(10, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(visible)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(20, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(30, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(x)
x = GlobalMaxPooling2D()(x)
output = Dense(Y_train.shape[1], activation='softmax', name = 'emotion', kernel_initializer=my_init)(x)
output2 = Dense(Y_train3.shape[1], activation='linear', kernel_initializer=my_init)(x)
model = Model(inputs=visible, outputs=[output, output2])

In [20]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 50, 50, 1)    0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 50, 50, 10)   100         input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 25, 25, 10)   0           conv2d_7[0][0]                   
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 25, 25, 20)   1820        max_pooling2d_5[0][0]            
__________________________________________________________________________________________________
max_poolin

In [21]:
model.compile(loss=['categorical_crossentropy', 'mse'],
              optimizer=Adam(clipnorm = 1.), metrics = {'emotion': 'accuracy'}, loss_weights = [1, 1])

In [22]:
model.fit(X_train, [Y_train, Y_train3],
          validation_data = (X_test, [Y_test, Y_test3]),
          batch_size = 64,
          epochs = 10,
          verbose = True,
          shuffle = False)

Train on 1982 samples, validate on 661 samples
Epoch 1/10
1982/1982 [==============================] - 7s 3ms/step - loss: 42.6993 - emotion_loss: 4.9381 - dense_2_loss: 37.7612 - emotion_acc: 0.2220 - val_loss: 2.3144 - val_emotion_loss: 1.8729 - val_dense_2_loss: 0.4415 - val_emotion_acc: 0.2678
Epoch 2/10
1982/1982 [==============================] - 6s 3ms/step - loss: 2.0071 - emotion_loss: 1.6848 - dense_2_loss: 0.3222 - emotion_acc: 0.3280 - val_loss: 1.8119 - val_emotion_loss: 1.5948 - val_dense_2_loss: 0.2171 - val_emotion_acc: 0.3374
Epoch 3/10
1982/1982 [==============================] - 6s 3ms/step - loss: 1.6957 - emotion_loss: 1.5108 - dense_2_loss: 0.1849 - emotion_acc: 0.3910 - val_loss: 1.6497 - val_emotion_loss: 1.4773 - val_dense_2_loss: 0.1725 - val_emotion_acc: 0.3782
Epoch 4/10
1982/1982 [==============================] - 7s 3ms/step - loss: 1.5818 - emotion_loss: 1.4177 - dense_2_loss: 0.1640 - emotion_acc: 0.4299 - val_loss: 1.5664 - val_emotion_loss: 1.3995 - va

In [23]:
pred = model.predict(X_test)[0]
y_pred = [np.argmax(p) for p in pred]
y_true = [np.argmax(p) for p in Y_test]

print classification_report(y_true, y_pred)

             precision    recall  f1-score   support

          0       0.69      0.34      0.45       149
          1       0.00      0.00      0.00        10
          2       0.71      0.07      0.13        71
          3       0.38      0.65      0.48        74
          4       0.52      0.88      0.65       163
          5       0.50      0.05      0.09        64
          6       0.53      0.72      0.61       130

avg / total       0.56      0.52      0.46       661



# CNN with  FACS and KEYPOINT

In [24]:
visible = Input(shape=input_shape)
x1 = Conv2D(10, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(visible)
x1 = MaxPooling2D(pool_size=(2, 2))(x1)
x2 = Conv2D(20, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(x1)
x2 = MaxPooling2D(pool_size=(2, 2))(x2)
x3 = Conv2D(30, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(x2)
x = GlobalMaxPooling2D()(x3)
output = Dense(Y_train.shape[1], activation='softmax', name = 'emotion', kernel_initializer=my_init)(x)
output2 = Dense(Y_train2.shape[1], activation='sigmoid', kernel_initializer=my_init)(GlobalMaxPooling2D()(x2))
output3 = Dense(Y_train3.shape[1], activation='linear', kernel_initializer=my_init)(GlobalMaxPooling2D()(x1))
model = Model(inputs=visible, outputs=[output, output2, output3])

In [25]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 50, 50, 1)    0                                            
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 50, 50, 10)   100         input_4[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_7 (MaxPooling2D)  (None, 25, 25, 10)   0           conv2d_10[0][0]                  
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 25, 25, 20)   1820        max_pooling2d_7[0][0]            
__________________________________________________________________________________________________
max_poolin

In [26]:
model.compile(loss=['categorical_crossentropy', 'binary_crossentropy', 'mse'],
              optimizer=Adam(clipnorm = 1.), metrics = {'emotion': 'accuracy'}, loss_weights = [1, 1e-1, 1e-1])

In [27]:
model.fit(X_train, [Y_train, Y_train2, Y_train3],
          validation_data = (X_test, [Y_test, Y_test2, Y_test3]),
          batch_size = 64,
          epochs = 10,
          verbose = True,
          shuffle = False)

Train on 1982 samples, validate on 661 samples
Epoch 1/10
1982/1982 [==============================] - 9s 5ms/step - loss: 35.3715 - emotion_loss: 10.2034 - dense_3_loss: 3.9324 - dense_4_loss: 247.7484 - emotion_acc: 0.2371 - val_loss: 16.8905 - val_emotion_loss: 10.1862 - val_dense_3_loss: 3.3374 - val_dense_4_loss: 63.7056 - val_emotion_acc: 0.2632
Epoch 2/10
1982/1982 [==============================] - 9s 4ms/step - loss: 12.6271 - emotion_loss: 10.2749 - dense_3_loss: 2.2526 - dense_4_loss: 21.2701 - emotion_acc: 0.2558 - val_loss: 10.1059 - val_emotion_loss: 9.7049 - val_dense_3_loss: 1.3860 - val_dense_4_loss: 2.6245 - val_emotion_acc: 0.2632
Epoch 3/10
1982/1982 [==============================] - 9s 5ms/step - loss: 7.7241 - emotion_loss: 7.3848 - dense_3_loss: 1.0280 - dense_4_loss: 2.3655 - emotion_acc: 0.2442 - val_loss: 3.5110 - val_emotion_loss: 3.2263 - val_dense_3_loss: 0.6811 - val_dense_4_loss: 2.1655 - val_emotion_acc: 0.3495
Epoch 4/10
1982/1982 [====================

In [28]:
pred = model.predict(X_test)[0]
y_pred = [np.argmax(p) for p in pred]
y_true = [np.argmax(p) for p in Y_test]

print classification_report(y_true, y_pred)

             precision    recall  f1-score   support

          0       0.75      0.56      0.64       149
          1       0.00      0.00      0.00        10
          2       0.79      0.27      0.40        71
          3       0.45      0.80      0.58        74
          4       0.67      0.88      0.76       163
          5       0.64      0.14      0.23        64
          6       0.61      0.78      0.68       130

avg / total       0.65      0.63      0.60       661



In [ ]:
# homework: to get 65% F1